In [1]:
from data import Data
import numpy as np

In [2]:
d = Data()
X_train,y_train = d.get_train()
X_cross, y_cross = d.get_dev()
X_test,y_test = d.get_test()

loaded 100
loaded 200
loaded 300
loaded 400
loaded 500
loaded 600
loaded 700
loaded 800
loaded 900
loaded 1000
loaded 1100
loaded 1200
loaded 1300
loaded 1400
loaded 1500
loaded 1600
loaded 1700
loaded 1800
loaded 1900
loaded 2000
loaded 2100
loaded 2200
loaded 2300
loaded 2400
loaded 2500
loaded 2600
loaded 2700
loaded 2800
loaded 2900
loaded 3000
loaded 3100
loaded 3200
loaded 3300
loaded 3400
loaded 3500
loaded 3600
loaded 3700
loaded 3800
loaded 3900
loaded 4000
loaded 4100
loaded 4200
loaded 4300
loaded 4400
loaded 4500
loaded 4600
loaded 4700
loaded 4800
loaded 4900
loaded 5000
loaded 5100
loaded 5200
loaded 5300
loaded 5400
loaded 5500
loaded 5600
loaded 5700
loaded 5800
loaded 5900
loaded 6000
loaded 6100
loaded 6200
loaded 6300
loaded 6400
loaded 6500
loaded 6600
loaded 6700
loaded 6800
loaded 6900
loaded 7000
loaded 7100
loaded 7200
loaded 7300
loaded 7400
loaded 7500
loaded 7600
loaded 7700
loaded 7800
loaded 7900
loaded 8000
loaded 8100
loaded 8200
loaded 8300
loaded 8400
l

In [7]:
X_train = X_train[:100]
y_train = y_train[:100]
print(X_train.shape)
print(y_train.shape)
print(X_cross.shape)
print(y_cross.shape)
print(X_test.shape)
print(y_test.shape)

(100, 240, 320, 3)
(100,)
(0, 240, 320, 3)
(0,)
(12921, 240, 320, 3)
(12921,)


In [20]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torch.nn.functional as F
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

In [9]:
class MyCustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __getitem__(self, index):
        img = self.X[index]
        label = self.y[index]
        return (img, label)

    def __len__(self):
        return self.X.shape[0]

In [10]:
NUM_TRAIN = X_train.shape[0]
train_dataset = MyCustomDataset(X_train, y_train)
loader_train = DataLoader(train_dataset, batch_size=10,sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

test_dataset = MyCustomDataset(X_test, y_test)
loader_test = DataLoader(test_dataset, batch_size=10)


In [11]:
def flatten(X):
    N = X.shape[0]
    return X.view(N,-1)

In [23]:
def check_accuracy(loader, model): 
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            scores = model(Variable(x.float()))
            _, preds = scores.max(1)
            num_correct += (preds == Variable(y.long())).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [22]:
print_every = 10
def train(model, optimizer, epochs=1):
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()
            scores = model(Variable(x.float()))
            loss = F.cross_entropy(scores, Variable(y.long()))
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                check_accuracy(loader_test, model)
                print()

In [18]:
class Flatten(nn.Module):
    def forward(self, x): 
        return flatten(x)

In [ ]:
hidden_layer_size = 3000
learning_rate = 1e-3
num_classes = 196

model = nn.Sequential(
    Flatten(),
    nn.Linear(3*240*320,hidden_layer_size),
    nn.ReLU(),
    nn.Linear(hidden_layer_size,num_classes)
    )
optimizer = optim.Adam(model.parameters(),lr = learning_rate)
train(model,optimizer)

Iteration 0, loss = 100.8970
